# Proyecto Integrado 2
---

### 1. Objetivo del proyecto 

Una empresa emergente vende productos alimenticios. Se debe investigar el comportamiento del usuario para la aplicación de la empresa.

---


### 2. Carga de dataset y procesamiento de datos 


In [1]:
# Importación de librerías neceserarias
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from plotly import graph_objects as go
import plotly.express as px
import re
from datetime import time

In [2]:
#Asignación de variable al dataset

def load_data():
    try:
        # Intentar cargar los datos de la forma local
        logs = pd.read_csv('logs_exp_us.csv', sep='\t') # Reemplaza con la ruta local real
        return logs
    except FileNotFoundError:
        # Si la carga local falla, intentar la carga desde la instancia de revisión
        try:
            logs = pd.read_csv('/datasets/logs_exp_us.csv', sep='\t')
            return logs
        except FileNotFoundError:
            print("No se pudo encontrar el archivo en ninguna de las ubicaciones.")
            return None

# Llamar a la función para cargar los datos
logs = load_data()

In [3]:
#Visualización de la información del dataset
logs.info()
logs.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244126 entries, 0 to 244125
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   EventName       244126 non-null  object
 1   DeviceIDHash    244126 non-null  int64 
 2   EventTimestamp  244126 non-null  int64 
 3   ExpId           244126 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 7.5+ MB


,EventName,DeviceIDHash,EventTimestamp,ExpId
57282,CartScreenAppear,5563901287895361556,1564750785,248
76418,OffersScreenAppear,3030959351435601703,1564808335,246
164869,CartScreenAppear,4705890939205361551,1565018860,246
2569,MainScreenAppear,1271154466483310782,1564611771,248
147410,CartScreenAppear,4257848154605915902,1564991934,247
35889,MainScreenAppear,6221557117339142225,1564687455,246
131096,OffersScreenAppear,7244344893422246761,1564933931,248
1462,MainScreenAppear,443148007654269818,1564586136,247
2468,MainScreenAppear,2380122902375934285,1564610705,248
138303,CartScreenAppear,3936827126930849525,1564947208,248


In [4]:
#Cambiar el nombre de las columnas 
new_names = {'EventName': 'Event_Name',
            'DeviceIDHash': 'Device_ID_Hash',
            'EventTimestamp': 'Event_Timestamp',
            'ExpId': 'Exp_Id'}

# Renombrar columnas usando el diccionario 
logs = logs.rename(columns=new_names)

logs.columns

Index(['Event_Name', 'Device_ID_Hash', 'Event_Timestamp', 'Exp_Id'], dtype='object')

In [10]:
logs['Event_Name'] = logs['Event_Name'].astype('category')

In [12]:
1 5 6 4 7 5 0 7 8 5


logs['Event_Timestamp'] = pd.to_datetime(logs['Event_Timestamp'])
logs.info()

TypeError: <class 'datetime.time'> is not convertible to datetime, at position 0